#Quiz
> * dataset은 370개의 feature로 이루어진 데이터
* 산탄데르 은행에서 캐글에 경연 의뢰한 데이터로 피처 이름은 임의로 처리됨
* TARGET 정보가 1이면 불만족, 0이면 만족
* https://www.kaggle.com/c/santander-customer-satisfaction/data

#1. 파일 읽기

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
import pandas as pd

train_file = '/content/drive/MyDrive/12. 앙상블/santander-customer-satisfaction/train.csv'
test_file = '/content/drive/MyDrive/12. 앙상블/santander-customer-satisfaction/test.csv'
df_train = pd.read_csv(train_file)
df_test = pd.read_csv(test_file)

df_train.head(2)

,ID,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,imp_op_var40_ult1,imp_op_var41_comer_ult1,imp_op_var41_comer_ult3,imp_op_var41_efect_ult1,imp_op_var41_efect_ult3,imp_op_var41_ult1,imp_op_var39_efect_ult1,imp_op_var39_efect_ult3,imp_op_var39_ult1,imp_sal_var16_ult1,ind_var1_0,ind_var1,ind_var2_0,ind_var2,ind_var5_0,ind_var5,ind_var6_0,ind_var6,ind_var8_0,ind_var8,ind_var12_0,ind_var12,ind_var13_0,ind_var13_corto_0,ind_var13_corto,ind_var13_largo_0,ind_var13_largo,ind_var13_medio_0,ind_var13_medio,ind_var13,...,saldo_medio_var5_ult1,saldo_medio_var5_ult3,saldo_medio_var8_hace2,saldo_medio_var8_hace3,saldo_medio_var8_ult1,saldo_medio_var8_ult3,saldo_medio_var12_hace2,saldo_medio_var12_hace3,saldo_medio_var12_ult1,saldo_medio_var12_ult3,saldo_medio_var13_corto_hace2,saldo_medio_var13_corto_hace3,saldo_medio_var13_corto_ult1,saldo_medio_var13_corto_ult3,saldo_medio_var13_largo_hace2,saldo_medio_var13_largo_hace3,saldo_medio_var13_largo_ult1,saldo_medio_var13_largo_ult3,saldo_medio_var13_medio_hace2,saldo_medio_var13_medio_hace3,saldo_medio_var13_medio_ult1,saldo_medio_var13_medio_ult3,saldo_medio_var17_hace2,saldo_medio_var17_hace3,saldo_medio_var17_ult1,saldo_medio_var17_ult3,saldo_medio_var29_hace2,saldo_medio_var29_hace3,saldo_medio_var29_ult1,saldo_medio_var29_ult3,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38,TARGET
0,1,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39205.17,0
1,3,2,34,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,1,0,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,300.0,122.22,300.0,240.75,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,49278.03,0


,ID,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,imp_op_var40_ult1,imp_op_var41_comer_ult1,imp_op_var41_comer_ult3,imp_op_var41_efect_ult1,imp_op_var41_efect_ult3,imp_op_var41_ult1,imp_op_var39_efect_ult1,imp_op_var39_efect_ult3,imp_op_var39_ult1,imp_sal_var16_ult1,ind_var1_0,ind_var1,ind_var2_0,ind_var2,ind_var5_0,ind_var5,ind_var6_0,ind_var6,ind_var8_0,ind_var8,ind_var12_0,ind_var12,ind_var13_0,ind_var13_corto_0,ind_var13_corto,ind_var13_largo_0,ind_var13_largo,ind_var13_medio_0,ind_var13_medio,ind_var13,...,saldo_medio_var5_hace3,saldo_medio_var5_ult1,saldo_medio_var5_ult3,saldo_medio_var8_hace2,saldo_medio_var8_hace3,saldo_medio_var8_ult1,saldo_medio_var8_ult3,saldo_medio_var12_hace2,saldo_medio_var12_hace3,saldo_medio_var12_ult1,saldo_medio_var12_ult3,saldo_medio_var13_corto_hace2,saldo_medio_var13_corto_hace3,saldo_medio_var13_corto_ult1,saldo_medio_var13_corto_ult3,saldo_medio_var13_largo_hace2,saldo_medio_var13_largo_hace3,saldo_medio_var13_largo_ult1,saldo_medio_var13_largo_ult3,saldo_medio_var13_medio_hace2,saldo_medio_var13_medio_hace3,saldo_medio_var13_medio_ult1,saldo_medio_var13_medio_ult3,saldo_medio_var17_hace2,saldo_medio_var17_hace3,saldo_medio_var17_ult1,saldo_medio_var17_ult3,saldo_medio_var29_hace2,saldo_medio_var29_hace3,saldo_medio_var29_ult1,saldo_medio_var29_ult3,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38
0,2,2,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,2.43,6.0,4.80,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40532.10
1,5,2,35,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,2.55,3.0,2.85,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,45486.72


#2. 데이터 전처리

##2.1 데이터 확인

In [3]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76020 entries, 0 to 76019
Columns: 371 entries, ID to TARGET
dtypes: float64(111), int64(260)
memory usage: 215.2 MB


> 371개의 feature 정보가 존재하고 수치정보로 이루어 진것을 알 수 있다.

In [ ]:
df_train.describe()

In [12]:
df_test.describe()

,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,imp_op_var40_ult1,imp_op_var41_comer_ult1,imp_op_var41_comer_ult3,imp_op_var41_efect_ult1,imp_op_var41_efect_ult3,imp_op_var41_ult1,imp_op_var39_efect_ult1,imp_op_var39_efect_ult3,imp_op_var39_ult1,imp_sal_var16_ult1,ind_var1_0,ind_var1,ind_var2_0,ind_var2,ind_var5_0,ind_var5,ind_var6_0,ind_var6,ind_var8_0,ind_var8,ind_var12_0,ind_var12,ind_var13_0,ind_var13_corto_0,ind_var13_corto,ind_var13_largo_0,ind_var13_largo,ind_var13_medio_0,ind_var13_medio,ind_var13,ind_var14_0,...,saldo_medio_var5_hace3,saldo_medio_var5_ult1,saldo_medio_var5_ult3,saldo_medio_var8_hace2,saldo_medio_var8_hace3,saldo_medio_var8_ult1,saldo_medio_var8_ult3,saldo_medio_var12_hace2,saldo_medio_var12_hace3,saldo_medio_var12_ult1,saldo_medio_var12_ult3,saldo_medio_var13_corto_hace2,saldo_medio_var13_corto_hace3,saldo_medio_var13_corto_ult1,saldo_medio_var13_corto_ult3,saldo_medio_var13_largo_hace2,saldo_medio_var13_largo_hace3,saldo_medio_var13_largo_ult1,saldo_medio_var13_largo_ult3,saldo_medio_var13_medio_hace2,saldo_medio_var13_medio_hace3,saldo_medio_var13_medio_ult1,saldo_medio_var13_medio_ult3,saldo_medio_var17_hace2,saldo_medio_var17_hace3,saldo_medio_var17_ult1,saldo_medio_var17_ult3,saldo_medio_var29_hace2,saldo_medio_var29_hace3,saldo_medio_var29_ult1,saldo_medio_var29_ult3,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38
count,75818.000000,75818.000000,75818.000000,75818.000000,75818.000000,75818.000000,75818.000000,75818.000000,75818.000000,75818.000000,75818.000000,75818.000000,75818.000000,75818.000000,75818.000000,75818.000000,75818.00000,75818.000000,75818.000000,75818.000000,75818.000000,75818.0,75818.0,75818.000000,75818.000000,75818.000000,75818.000000,75818.000000,75818.000000,75818.000000,75818.000000,75818.000000,75818.000000,75818.000000,75818.000000,75818.000000,75818.000000,75818.000000,75818.000000,75818.000000,...,75818.000000,75818.000000,75818.000000,75818.000000,75818.000000,75818.000000,75818.000000,7.581800e+04,7.581800e+04,7.581800e+04,7.581800e+04,75818.000000,75818.000000,75818.000000,75818.000000,7.581800e+04,75818.000000,7.581800e+04,75818.000000,75818.000000,75818.0,75818.000000,75818.000000,75818.000000,75818.000000,75818.000000,75818.000000,75818.000000,75818.0,75818.000000,75818.000000,75818.000000,75818.000000,75818.000000,75818.000000,75818.000000,75818.000000,75818.000000,75818.000000,7.581800e+04
mean,2.779432,33.138832,83.164329,74.312894,123.136448,4.578517,7.666855,0.470645,0.672444,3.635893,69.734376,115.469593,67.439982,110.967136,137.214620,67.910627,111.63958,140.850513,4.917151,0.012108,0.003917,0.0,0.0,0.957002,0.663747,0.000092,0.000026,0.033686,0.029887,0.068612,0.045359,0.053206,0.043486,0.042154,0.010763,0.010578,0.000066,0.000066,0.051874,0.024414,...,902.370806,1056.867678,1060.014504,60.764337,9.361781,118.186622,104.805798,4.359742e+03,7.048762e+02,6.059647e+03,4.745336e+03,3581.184666,501.124806,4909.473875,3869.986824,8.167134e+02,176.592698,9.869169e+02,771.302026,0.199158,0.0,1.218142,0.708650,36.323577,6.204746,57.528068,47.688547,0.138711,0.0,0.195140,0.160595,12.438559,1.327408,17.469991,12.674349,63.597839,11.404505,95.973025,70.504319,1.173863e+05
std,10.118252,12.932000,1694.873886,364.211245,606.431562,133.383326,239.701416,34.028605,42.336668,129.089127,330.563913,540.054605,517.066406,726.368848,697.823263,521.377495,732.61944,717.878305,322.535540,0.109369,0.062466,0.0,0.0,0.202853,0.472430,0.009608,0.005136,0.180421,0.170278,0.252794,0.208091,0.224446,0.203949,0.200941,0.103184,0.102304,0.008121,0.008121,0.221775,0.154331,...,7477.578055,9787.585269,8138.430011,1697.257696,510.104537,2240.333363,1815.886791,4.549291e+04,1.184441e+04,5.116131e+04,4.000231e+04,25634.05725

> 수치 데이터 확인 결과 -999999라는 이상치를 확인했다. 뿐만 아니라 각 feature 별로 4분위수의 범위를 한참 벗어나는 수치들이 나타나기 때문에 대부분 이상치로 보여 지기는 한다. 예를 들어 var15의 경우 4분위수는 모두 0으로 나타나지만 최대값만 21000으로 나타난다. 현재 데이터는 고객 만족에 대한 데이터이고 만약 var15의 값이 설문에 대한 내용이라고 판단된다면 최대값이 5점 최소값이 0점이라는 가정하에 모두 최고점을 주었다고 생각할 수도 있다.(물론 가정이다). 어쨌든 현재 데이터들이 정확치 않은 상황에서 이상치 데이터를 변환하는 것보다는 있는 그대로의 값으로 사용하는 것이 좋을 듯 하다.

##2.2 이상치 처리

In [ ]:
df_train['var3'].value_counts().sort_index()

In [ ]:
df_test['var3'].value_counts().sort_index()

> 현재 데이터가 -999999를 제외한 모든 값들이 0이상의 양수로 표현되며 순차적인 데이터로 보인다. -999999의 의미가 정확히 무엇인지는 알 수없다. 예를 들어 설문으로 생각한다면 무응답 데이터로 판단할 수도 있고 에러나 예외 관련된 데이터일 수도 있다. -999999의 데이터를 전체 흐름에 벗어나지 않도록 수정한다면 -1정도로 하여 간격을 맞추는 것도 한 방법일 것이다.

In [ ]:
df_train['var3'].replace(-999999, -1, inplace=True)
df_test['var3'].replace(-999999, -1, inplace=True)
df_train['var3'].value_counts().sort_index()

> -999999의 데이터 개수가 116개 였고 2를 제외한 다른 값들을 보더라도 100개 안밖으로 나타남으로 다른 데이터에 영향을 미치지 않도록 -999999를 -1로 변경한다.

##2.3 불필요한 feature 처리

In [ ]:
df_train.describe()

In [ ]:
df_train.drop('ID', axis=1, inplace=True)
df_test.drop('ID', axis=1, inplace=True)
df_train.head(2)

#3. 모델 만들기

##3.1 데이터 분리

In [15]:
df_labels = df_train['TARGET']
df_data = df_train.drop(columns='TARGET', axis=1)

In [16]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df_data, df_labels, test_size=0.3, random_state=62)

##3.2 XGBoost

In [18]:
from xgboost import XGBClassifier

# n_estimators는 500개의 분류기 사용
xgb_clf = XGBClassifier(n_estimators=500)
# early_stopping_rounds은 데이터 검증 결과가 100회 동안 범위를 유지하면 중지함.
# eval_metric은 성능 평가 지표를 auc를 이용함
# auc는 이진 분류기일 경우 사용되며 현재 고객의 [만족, 불만족] 두 가지를 평가하는 것이기 때문에 사용함.
# eval_set은 성능 평가 데이터셋
xgb_clf.fit(X_train, y_train, early_stopping_rounds=100, eval_metric='auc', eval_set=[(X_test, y_test)])

[0]	validation_0-auc:0.788351
Will train until validation_0-auc hasn't improved in 100 rounds.
[1]	validation_0-auc:0.792659
[2]	validation_0-auc:0.795539
[3]	validation_0-auc:0.804695
[4]	validation_0-auc:0.804109
[5]	validation_0-auc:0.803495
[6]	validation_0-auc:0.805019
[7]	validation_0-auc:0.80446
[8]	validation_0-auc:0.803224
[9]	validation_0-auc:0.804659
[10]	validation_0-auc:0.804867
[11]	validation_0-auc:0.804273
[12]	validation_0-auc:0.80433
[13]	validation_0-auc:0.80613
[14]	validation_0-auc:0.806088
[15]	validation_0-auc:0.806062
[16]	validation_0-auc:0.806922
[17]	validation_0-auc:0.806938
[18]	validation_0-auc:0.807515
[19]	validation_0-auc:0.810904
[20]	validation_0-auc:0.810939
[21]	validation_0-auc:0.814049
[22]	validation_0-auc:0.814023
[23]	validation_0-auc:0.814269
[24]	validation_0-auc:0.816089
[25]	validation_0-auc:0.817959
[26]	validation_0-auc:0.820136
[27]	validation_0-auc:0.819173
[28]	validation_0-auc:0.819297
[29]	validation_0-auc:0.821036
[30]	validation_0-

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=500, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [ ]:
xgb_clf.predict_proba(X_test)

In [24]:
from sklearn.metrics import roc_auc_score

# predict_proba는 확률 정보를 갖게 되며 [:, 1]은 두 가지 값 중 1의 값을 사용한다는 의미이다.
# 보통 1이 될 확률을 구하게 된다.
xgb_roc_score = roc_auc_score(y_test, xgb_clf.predict_proba(X_test)[:, 1])
xgb_roc_score

0.8341040578504227

##3.3 하이퍼 파라미터

In [26]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier

xgb_clf = XGBClassifier(n_estimators=100)

# min_child_weight는 min_samples_leaf 비슷한 값으로 과적합 해결
# colsample_bytree는 max_features와 유사하며 feature가 많을 경우 과적합 조절을 위해 사용, 확률 데이터
params = {
    'max_depth':[3, 6],
    'min_child_weight':[1, 3],
    'colsample_bytree':[0.5, 0.75]
}

grid_cv = GridSearchCV(xgb_clf, param_grid = params, cv=3)
grid_cv.fit(X_train, y_train, early_stopping_rounds=30, eval_metric='auc', eval_set=[(X_test, y_test)])

print('최적의 파라미터 : ', grid_cv.best_params_)
xgb_roc_score = roc_auc_score(y_test, grid_cv.predict_proba(X_test)[:, 1])
print('AUC : ', xgb_roc_score)

[0]	validation_0-auc:0.695022
Will train until validation_0-auc hasn't improved in 30 rounds.
[1]	validation_0-auc:0.777975
[2]	validation_0-auc:0.779777
[3]	validation_0-auc:0.78152
[4]	validation_0-auc:0.78114
[5]	validation_0-auc:0.779589
[6]	validation_0-auc:0.775586
[7]	validation_0-auc:0.774862
[8]	validation_0-auc:0.772166
[9]	validation_0-auc:0.773003
[10]	validation_0-auc:0.78578
[11]	validation_0-auc:0.78453
[12]	validation_0-auc:0.79265
[13]	validation_0-auc:0.791754
[14]	validation_0-auc:0.796619
[15]	validation_0-auc:0.798908
[16]	validation_0-auc:0.797294
[17]	validation_0-auc:0.800995
[18]	validation_0-auc:0.805757
[19]	validation_0-auc:0.807919
[20]	validation_0-auc:0.807355
[21]	validation_0-auc:0.806726
[22]	validation_0-auc:0.809641
[23]	validation_0-auc:0.81289
[24]	validation_0-auc:0.809742
[25]	validation_0-auc:0.815774
[26]	validation_0-auc:0.813977
[27]	validation_0-auc:0.817042
[28]	validation_0-auc:0.816359
[29]	validation_0-auc:0.819082
[30]	validation_0-auc:

In [28]:
import numpy as np

# n_estimators는 1000으로 증가시키고, learning_rate=0.02로 감소
# reg_alpha : 피처 개수가 많을 때 적용을 검토, 클수록 과적합 감소 효과
xgb_clf = XGBClassifier(n_estimators=1000, learning_rate=0.02, max_depth=3, 
                        min_child_weight=1, colsample_bytree=0.5, reg_alpha=0.03)

# 조기 중단 파라미터 값은 200으로 설정
xgb_clf.fit(X_train, y_train, early_stopping_rounds=200, eval_metric="auc", eval_set=[(X_test, y_test)])

xgb_roc_score = roc_auc_score(y_test, xgb_clf.predict_proba(X_test)[:, 1])
print(f'ROC-AUC: {np.round(xgb_roc_score, 4)}')

[0]	validation_0-auc:0.692231
Will train until validation_0-auc hasn't improved in 200 rounds.
[1]	validation_0-auc:0.777911
[2]	validation_0-auc:0.786793
[3]	validation_0-auc:0.785302
[4]	validation_0-auc:0.782662
[5]	validation_0-auc:0.779977
[6]	validation_0-auc:0.777602
[7]	validation_0-auc:0.776389
[8]	validation_0-auc:0.775328
[9]	validation_0-auc:0.775437
[10]	validation_0-auc:0.783888
[11]	validation_0-auc:0.782643
[12]	validation_0-auc:0.787659
[13]	validation_0-auc:0.788756
[14]	validation_0-auc:0.792706
[15]	validation_0-auc:0.793255
[16]	validation_0-auc:0.793315
[17]	validation_0-auc:0.795564
[18]	validation_0-auc:0.796108
[19]	validation_0-auc:0.796904
[20]	validation_0-auc:0.796835
[21]	validation_0-auc:0.796472
[22]	validation_0-auc:0.796731
[23]	validation_0-auc:0.797196
[24]	validation_0-auc:0.797078
[25]	validation_0-auc:0.801208
[26]	validation_0-auc:0.79734
[27]	validation_0-auc:0.801087
[28]	validation_0-auc:0.796502
[29]	validation_0-auc:0.799876
[30]	validation_